In [ ]:
from google.colab import files

# Upload your CSV file
uploaded = files.upload()


Saving indiancrop_dataset.csv to indiancrop_dataset.csv


In [ ]:
!ls


indiancrop_dataset.csv	sample_data


In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv("indiancrop_dataset.csv")

# Show first 5 rows
data.head()


N_SOIL  P_SOIL  K_SOIL  TEMPERATURE   HUMIDITY        ph    RAINFALL  \
0      90      42      43    20.879744  82.002744  6.502985  202.935536   
1      85      58      41    21.770462  80.319644  7.038096  226.655537   
2      60      55      44    23.004459  82.320763  7.840207  263.964248   
3      74      35      40    26.491096  80.158363  6.980401  242.864034   
4      78      42      42    20.130175  81.604873  7.628473  262.717340   

                 STATE  CROP_PRICE  CROP  
0  Andaman and Nicobar        7000  Rice  
1  Andaman and Nicobar        5000  Rice  
2  Andaman and Nicobar        7000  Rice  
3  Andaman and Nicobar        7000  Rice  
4  Andaman and Nicobar      120000  Rice

In [ ]:
# Fill numeric missing values with mean
numeric_cols = ['N_SOIL','P_SOIL','K_SOIL','TEMPERATURE','HUMIDITY','ph','RAINFALL','CROP_PRICE']
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Fill categorical missing values with mode
categorical_cols = ['STATE','CROP']
for col in categorical_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)


/tmp/ipython-input-1700649850.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)


In [ ]:
# Convert categorical columns into numeric using one-hot encoding
data = pd.get_dummies(data, columns=['STATE','CROP'])


In [ ]:
X = data.drop("CROP_PRICE", axis=1)  # Features
y = data["CROP_PRICE"]               # Target


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features (important for Neural Networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Train model
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest MSE:", mse_rf)
print("Random Forest R2 Score:", r2_rf)


Random Forest MSE: 7094004.654533921
Random Forest R2 Score: -0.03668083798042776


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build model
nn_model = Sequential()
nn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(1))  # Regression output

# Compile
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train
history = nn_model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=16)


Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 20391692.0000 - mae: 2681.4387 - val_loss: 14953760.0000 - val_mae: 2639.6013
Epoch 2/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20704974.0000 - mae: 2629.0852 - val_loss: 13821196.0000 - val_mae: 2437.1025
Epoch 3/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25753966.0000 - mae: 2234.6787 - val_loss: 11241935.0000 - val_mae: 1965.9812
Epoch 4/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13207985.0000 - mae: 1748.9661 - val_loss: 8576652.0000 - val_mae: 1649.4004
Epoch 5/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12684578.0000 - mae: 1548.1592 - val_loss: 7476097.5000 - val_mae: 1649.3026
Epoch 6/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11047877.0000 - mae: 1646.9005 - val_loss: 7226006.5000 - val_mae: 1679.3062
Epoch 7/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9506303.0000 - mae: 1587.7108 - val_loss: 7164405.0000 - val_mae: 1696.2051
Epoch 8/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss:

In [ ]:
y_pred_nn = nn_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
print("Neural Network MSE:", mse_nn)
print("Neural Network R2 Score:", r2_nn)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Neural Network MSE: 7086843.0
Neural Network R2 Score: -0.03563427925109863


In [ ]:
import joblib
joblib.dump(rf_model, "crop_yield_rf_model.pkl")


['crop_yield_rf_model.pkl']

In [ ]:
nn_model.save("crop_yield_nn_model.h5")


In [ ]:
# Random Forest
rf_model = joblib.load("crop_yield_rf_model.pkl")

# Neural Network
from tensorflow.keras.models import load_model
nn_model = load_model("crop_yield_nn_model.h5")


In [ ]:
import numpy as np

# Example: N_SOIL, P_SOIL, K_SOIL, TEMPERATURE, HUMIDITY, ph, RAINFALL + categorical one-hot encoded columns
# You need to match the same columns as X
new_data = np.array([[80, 50, 45, 25, 75, 7, 210] + [0]*(X_train.shape[1]-7)])  # adjust categorical columns

# Scale new data
new_data = scaler.transform(new_data)

# Predict
predicted_price_rf = rf_model.predict(new_data)
predicted_price_nn = nn_model.predict(new_data)

print("Predicted Crop Price (RF):", predicted_price_rf[0])
print("Predicted Crop Price (NN):", predicted_price_nn[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted Crop Price (RF): 2554.73
Predicted Crop Price (NN): [6044.332]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
